In [56]:
import sys
import os
import platform

import time

# suppress python warning messages
import warnings
warnings.filterwarnings("ignore")

# Langchain depedencies
from langchain import OpenAI
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

## Import data loaders for a specific doc or Directory
from langchain.document_loaders import TextLoader, DirectoryLoader, UnstructuredWordDocumentLoader, UnstructuredPDFLoader

## Import embeddings
from langchain.embeddings.openai import OpenAIEmbeddings

## Import Index with vector DBS/stores
from langchain.vectorstores import FAISS

## Import character/text splitter
from langchain.text_splitter import CharacterTextSplitter


## Import data loader from local file
from data_loader import data_loader



In [57]:
## vars
OPENAI_API_KEY = "INSERT KEY"
API_KEY_CONFIRMED = ""
# vectorstore path
vectorstore_path = "./FAISS_DB/faiss_index/"

In [58]:
# set embeddings
def embeddings():
    embeddings = OpenAIEmbeddings(openai_api_key=f"{OPENAI_API_KEY}")
    
    return embeddings

In [59]:
def initialize_vectorstore(OPENAI_API_KEY = None):
    if os.path.exists(f'{vectorstore_path}'):
        dir = os.listdir(f"{vectorstore_path}")
        if len(dir) == 0:
            # Creating new db/index - importing func from data_loader.py
            db = data_loader()
        else:
            # if files already exist, just load the existing db in the path
            db = FAISS.load_local(f"{vectorstore_path}", embeddings())
    
    return db

In [60]:
##  expose this index in a retriever interface.
def retriever():
    retriever = initialize_vectorstore().as_retriever(search_kwargs={"k":2})
    return retriever

In [61]:
# qa chain - (langchain)
def qa_chain():
    qa = RetrievalQA.from_chain_type(
        llm=OpenAI(),
        chain_type="stuff",
        retriever=retriever()
    )
    return qa


In [62]:
initialize_vectorstore()

In [63]:
query_prompt = "What is Jatin's AWS exam score?"

In [64]:
result = qa_chain().run(query_prompt)
print(result)

 Jatin's AWS exam score is 798.
